# Writing Functions in Python

## Introduction

These are my notes for DataCamp's course [_Writing Functions in Python_](https://www.datacamp.com/courses/writing-functions-in-python).

This course is presented by Shayne Miel, Director of Software Engineering at American Efficient. Collaborators are Hillary Green-Lerman and Becca Robbins.

Prerequisite:

- [_Python Data Science Toolbox (Part 2)_](../Python%20Data%20Science%20Toolbox%20Part%202/Python%20Data%20Science%20Toolbox%20Part%202.ipynb)

This course is part of these tracks:

- Data Engineer with Python
- Data Scientist with Python
- Python Programmer
- Python Programming

There are no datasets for this course.

## Versions

The course's IDE uses Python "3.9.7 (default, Sep 10 2021, 00:03:59) \[GCC 7.5.0]".

This notebook is being written using Python 3.11.2.

## Data Set

| File | Description |
| :--- | :----|
| alice.txt | The complete text of _Alice in Wonderland_ |

## Resources

### Docstrings
- [Python PEP 257 - Docstring Conventions](https://peps.python.org/pep-0257/)
- [reStructuredText Markup](https://devguide.python.org/documentation/markup/)
- [DataCamp Docstrings Tutorial](https://www.datacamp.com/tutorial/docstrings-python)
- [Numpy Style Guide](https://numpydoc.readthedocs.io/en/latest/format.html)
- [Google Style Guide for Comments and Docstrings](https://google.github.io/styleguide/pyguide.html#38-comments-and-docstrings)

### inspect Module
- [inspect - Inspect live objects](https://docs.python.org/3/library/inspect.html)

### functools Module
- [functools — Higher-order functions and operations on callable objects](https://docs.python.org/3/library/functools.html)

## Imports

Imports are gathered here for clarity and convenience.

In [ ]:
import contextlib
import functools
import inspect
import random

import numpy as np
import pandas as pd
import time

## Best Practices

### Docstrings

#### Example Docstring (Demonstration)

```python
def split_and_stack(df, new_names):
    """
    Split a DataFrame's columns into two halves and then stack
    them vertically, returning a new DataFrame with 'new_names' as the
    column names.
    
    Args:
        df (DataFrame): The DataFrame to split.
        new_names (iterable of str): The column names for the new DataFrame
    
    Returns:
        DataFrame
    """
    half = int(len(df.columns) / 2)
    left = df.iloc[:, :half]
    right = df.iloc[:, half:]
    return pd.DataFrame(
        data=np.vstack([left.values, right.values]),
        columns=new_names
    )
```

#### Anatomy of a Docstring (Demonstration)

```python
def function_name(arguments):
    """
    Description of what the function does.
    
    Description of the arguments, if any.
    
    Description of the return values, if any.
    
    Description of errors raised, if any.
    
    Optional extra notes or examples of usage.
    """
```

There are four major docstring formats:
- Google Style
- Numpydoc
- reStructured Text
- EpyText

This course focuses on Google Style and Numpydoc.

#### Google Style Docstrings (Demonstration)

Google style docstrings are used by this course because the format is more
compact.

```python
def function(arg_1, arg_2=42):
    """
    Imperative description of what the function does.
    
    Args:
        arg_1 (str): Description of arg_1 that can break into the next line
            if needed.
        arg_2 (int, optional): Write optional when an argument has a default
            value
    
    Returns:
        bool: Optional description of the return value
        Extra lines are not indented
    
    Raises:
        ValueError: Include any error types that the function intentionally
            raises
    
    Notes:
        See https://www.datacamp.com/tutorial/docstrings-python
        for more information.
    """
```

#### Numpydoc Docstrings (Demonstration)

Numpydoc docstrings are the most common in the scientific community.

```python
def function(arg_1, arg_2=42):
    """
    Imperative description of what the function does.
    
    Parameters
    ----------
    arg_1 : expected type of arg_1
        Description of arg_1
    arg_2 : int, optional
        Write optional when an argument has a default value.
        Default=42.
        
    Returns
    -------
    The type of the return value
        Can include a description of the return value.
        Replace "Returns" with "Yields" if this function is a generator.
    """
```

#### Retrieving Docstrings (Demonstration)

In [ ]:
def the_answer():
    """
    Return the answer to life,
    the universe, and everything.

    Returns:
        int
    """
    return 42
print(the_answer.__doc__)
# Remove leading spaces.
print(inspect.getdoc(the_answer))

#### Crafting a Docstring (Exercise)

In [ ]:
def count_letter(content, letter):
    """
    Count the number of times `letter` appears in `content`.

    Args:
        content (str): The string to search.
        letter (str): The letter to search for.
    
    Returns:
        int
    
    Raises:
        ValueError: If `letter` is not a one-character string.
    """
    if (not isinstance(letter, str)) or len(letter) != 1:
        raise ValueError('`letter` must be a single character string.')
    return len([char for char in content if char == letter])

#### Retrieving Docstrings (Exercise)

In [ ]:
# Display the unprocessed docstring.
docstring = count_letter.__doc__
border = '#' * 28
print('{}\n{}\n{}'.format(border, docstring, border))

# Use inspect.getdoc to remove leading and trailing blank lines and leading
# white space from the docstring.
docstring = inspect.getdoc(count_letter)
border = '#' * 28
print('{}\n{}\n{}'.format(border, docstring, border))

def build_tooltip(function):
    """
    Create a tooltip for any function that shows the
    function's docstring.
    
    Args:
        function (callable): The function we want a tooltip for.
    
    Returns:
        str
    """
    # Get the docstring for the function argument by using inspect.
    docstring = inspect.getdoc(function)
    border = "#" * 28
    return "{}\n{}\n{}".format(border, docstring, border)

print(build_tooltip(count_letter))
print(build_tooltip(range))
print(build_tooltip(print))
print(build_tooltip(build_tooltip))
print(build_tooltip(inspect.getdoc))

#### Docstrings to the Rescue! (Exercise)

In [ ]:
# This was an exercise in looking at docstrings.
print(np.histogram.__doc__)

### DRY and "Do One Thing"

#### Don't Repeat Yourself (Demonstration)

This code repeats itself, and it contains an error in the last code block (`### yikes! ###`). Code like this is difficult to maintain because any change in the algorithm must be made at three separate locations.

```python
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA

# Analyze the training data.
train = pd.read_csv("train.csv")
train_y = train["labels"].values
train_X = train[col for col in train.columns if col != "labels"].values
train_pca = PCA(n_components=2).fit_transform(train_X)
plt.scatter(train_pca[:, 0], train_pca[:, 1])

# Analyze the validation data.
val = pd.read_csv("validation.csv")
val_y = val["labels"].values
val_X = val[col for col in val.columns if col != "labels"].values
val_pca = PCA(n_components=2).fit_transform(val_X)
plt.scatter(val_pca[:, 0], val_pca[:, 1])

# Analyze the test data.
test = pd.read_csv("test.csv")
test_y = test["labels"].values
test_X = test[col for col in test.columns if col != "labels"].values
test_pca = PCA(n_components=2).fit_transform(train_X) ### yikes! ###
plt.scatter(test_pca[:, 0], test_pca[:, 1])
```

This is where a function is useful for eliminating repeated code. The function
does the desired work and returns the x and y values for each dataset for
further use. (Note that we provide a well-formatted docstring for the
function.)

```python
def load_and_plot(path):
    """
    Load a dataset and plot the first two principal components.
    
    Args:
        path (str): The location of the CSV file.
    
    Returns:
        tuple of ndarray: (features, labels)
    """
    data = pd.read_csv(path)
    data_y = data["labels"].values
    data_X = data[col for col in data.columns if col != "labels"].values
    data_pca = PCA(n_components=2).fit_transform(data_X)
    plt.scatter(data_pca[:, 0], data_pca[:, 1])
    return data_X, data_y

train_X, train_y = plot_pca("train.csv")
val_X, val_y = plot_pca("validation.csv")
test_X, test_y = plot_pca("test.csv")
```

At this point, this function violates another software engineering principle:
It does not do just one thing. The function does three things:

1) it loads data
2) it transforms data
3) it plots data

Here, the course creates two functions that decouple data loading from data transformation and plotting.

```python
def load_data(path):
    """
    Load a dataset and return the x and y values.
    
    Args:
        path (str): The location of the CSV file.
    
    Returns:
        tuple of ndarray: (features, labels)
    """
    data = pd.read_csv(path)
    data_y = data["labels"].values
    data_X = data[col for col in data.columns if col != "labels"].values
    return data_X, data_y

def plot_data(data_X):
    """
    Plot the first two principal components of a matrix.
    
    Args:
        data_X (numpy.ndarray): The data to plot.
    """
    data_pca = PCA(n_components=2).fit_transform(data_X)
    plt.scatter(data_pca[:, 0], data_pca[:, 1])
```

When writing functions that do one thing, the code becomes:
- more flexible
- more easily understood
- simpler to test
- simpler to debug
- easier to change

Shayne Miel recommends reading _Refactoring: Improving the Design of Existing Code (2nd Edition)_ by Martin Fowler.

By the way, reading this code inspired me to look into how to do principle component analysis using PCA. See [Principal Component Analysis of Breast Cancer Dataset](../Principal%20Component%20Analysis%20in%20Python/Principal%20Component%20Analysis%20of%20Breast%20Cancer%20Dataset.ipynb).

#### Extract a Function (Exercise)

Create a function that standardizes the values in a column, and use it on four columns of a DataFrame.

```python
def standardize(column):
    """
    Standardize the values in a column.

    Args:
        column (pandas Series): The data to standardize.

    Returns:
        pandas Series: the values as z-scores
    """
    # Finish the function so that it returns the z-scores
    z_score = (column - column.mean()) / column.std()
    return z_score

# Use the standardize() function to calculate the z-scores
df['y1_z'] = standardize(df.y1_gpa)
df['y2_z'] = standardize(df.y2_gpa)
df['y3_z'] = standardize(df.y3_gpa)
df['y4_z'] = standardize(df.y4_gpa)
```

#### Split Up a Function (Exercise)

Split up the original function, which calculates both mean and median and returns them, into two functions, each of which does one thing.

```python
def mean(values):
    """
    Return the mean of a sorted list of values.
    
    Args:
        values (iterable of float): A list of numbers
    
    Returns:
        float
    """
    mean = sum(values) / len(values)
    return mean

def median(values):
    """
    Return the median of a sorted list of values.
    
    Args:
        values (iterable of float): A list of numbers
    
    Returns:
        float
    """
    midpoint = int(len(values) / 2)
    if len(values) %2 == 0:
        median = (values[midpoint - 1] + values[midpoint]) / 2
    else:
        median = values[midpoint]
    
    return median
```

### Pass by Assignment

A list is mutable, but an integer is immutable.

See [Pass-by-value, reference, and assignment](https://mathspp.com/blog/pydonts/pass-by-value-reference-and-assignment).

In [ ]:
# Pass by reference (using a pointer).
def foo(x):
    x[0] = 99
my_list = [1, 2, 3]
print(my_list)
foo(my_list)
print(my_list)
print()

# Pass by value?
def bar(x):
    x = x + 90
my_var = 3
print(my_var)
bar(my_var)
print(my_var)
print()

# a and b refer to the same list.
a = [1, 2, 3]
print(a)
b = a
a.append(4)
print(b)
b.append(5)
print(a)

Immutable data types:
- int
- float
- bool
- string
- bytes
- tuple
- frozenset
- None

Mutable data types:
- list
- dict
- set
- bytearray
- objects
- functions
- almost everything else!

#### Mutable Default Arguments Are Dangerous! (Demonstration)

See this example for why you shouldn't set a default to an empty list or another mutable object.

In [ ]:
def foo(var=[]):
    var.append(1)
    return var
print(foo())
print(foo())
print()

# This is the correct way.
def foo2(var=None):
    if var is None:
        var = []
    var.append(1)
    return var
print(foo2())
print(foo2())

#### Mutable or Immutable? (Exercise)

The following function adds a mapping between a string and the lowercase version of that string to a dictionary. What do you expect the values of d and s to be after the function is called?

In [ ]:
def store_lower(_dict, _string):
    """
    Add a mapping between `_string` and a lowercased version of `_string` to
    `_dict`

    Args:
        _dict (dict): The dictionary to update.
        _string (str): The string to add.
    """
    orig_string = _string
    _string = _string.lower()
    _dict[orig_string] = _string

# A dictionary is a mutable object, but a string is immutable.
d = {}
s = 'Hello'

store_lower(d, s)
print(d,)
print(s)

#### Best Practice for Default Arguments (Exercise)

Avoid using a mutable default argument.

In [ ]:
def better_add_column(values, df=None):
    """
    Add a column of `values` to a DataFrame `df`.
    The column will be named "col_<n>", where "n" is
    the numerical index of the column.
    
    Args:
        values (iterable): The values of the new column
        df (DataFrame, optional): The DataFrame to update.
            If no DataFrame is passed, one is created by default.
    
    Returns:
        DataFrame
    """
    if df is None:
        df = pd.DataFrame()
    df["col_{}".format(len(df.columns))] = values
    return df

df = better_add_column([1, 2, 3], None)
df = better_add_column([4, 5, 6], df)
print(df.head())

## Context Managers

### Using Context managers

#### Examples (Demonstration)

A context manager sets up a contex, runs your code, and removes the context. Here, `open()` sets up a context by opening a file, lets you run any code you want on that file, and removes the context by closing the file.

```python
with open("my_file.txt") as my_file:
    text = my_file.read()
    length = len(text)
print("The file is {} characters long.".format(length))
```

Using `with` creates a compound statement, which is used as shown below:

```python
with <context-manager>(<args>):
    # Run your code here.
    # This code is running "inside the context"
# This code runs after the context is removed.
```

Some context managers return a value. Use `as` to capture that value. For example, `with open()` returns a file handle, which can be used within the context.

#### Reading a File (Exercise)

How many times does the word "cat" or "cats" appear in _Alice in Wonderland_?

In [ ]:
# The context manager closes the file for you.
with open("alice.txt") as file:
    text = file.read()
n = 0
for word in text.split():
    if word.lower() in ["cat", "cats"]:
        n += 1
print('Lewis Carroll used the word "cat" {} times.'.format(n))


#### Using a Timer Context Manager (Exercise)

I used the IPython shell to obtain the code for the functions used to support this example. It was amusing and instructive to do this.

    In [6]: import inspect
    In [7]: print(inspect.getsource(get_image_from_instagram))
    def get_image_from_instagram():
      return np.random.rand(84, 84)
    
    In [8]: print(inspect.getsource(process_with_numpy))
    def process_with_numpy(p):
      _process_pic(0.1521)
    
    In [9]: print(inspect.getsource(process_with_pytorch))
    def process_with_pytorch(p):
      _process_pic(0.0328)
    
    In [10]: print(inspect.getsource(_process_pic))
    def _process_pic(n_sec):
      print('Processing', end='', flush=True)
      for i in range(10):
        print('.', end='' if i < 9 else 'done!\n', flush=True)
        time.sleep(n_sec)
    
    In [11]: print(inspect.getsource(timer))
    @contextlib.contextmanager
    def timer():
      """Time how long code in the context block takes to run."""
      t0 = time.time()
      try:
        yield
      except:
        raise
      finally:
        t1 = time.time()
      print('Elapsed: {:.2f} seconds'.format(t1 - t0))

In [ ]:
# This code supports the simulation.
def get_image_from_instagram():
    return np.random.rand(84, 84)

def process_with_numpy(p):
    _process_pic(0.1521)

def process_with_pytorch(p):
    _process_pic(0.0328)

def _process_pic(n_sec):
    print('Processing', end='', flush=True)
    for i in range(10):
        print('.', end='' if i < 9 else 'done!\n', flush=True)
        time.sleep(n_sec)

@contextlib.contextmanager
def timer():
    """
    Time how long code in the context block takes to run.
    """
    t0 = time.time()
    try:
        yield
    except:
        raise
    finally:
        t1 = time.time()
    print('Elapsed: {:.2f} seconds'.format(t1 - t0))

##############################################################################
# Exercise code.
image = get_image_from_instagram()
with timer():
    print('Numpy version')
    process_with_numpy(image)
print()
with timer():
    print('Pytorch version')
    process_with_pytorch(image)

You may have noticed there was no `as <variable name>` at the end of the `with` statement in the `timer()` context manager. That is because `timer()` is a context manager that does not return a value, so the `as <variable name>` at the end of the `with` statement isn't necessary. In the next lesson, you'll learn how to write your own context managers like `timer()`.

### Writing Context Managers

There are two ways to define a context manager.
- Class-based
- Function-based

Since this is a course on writing functions, Shayne Miel has chosen to demonstrate the function-based approach.

#### How to Create a Context Manager (Demonstration)

```python
@contextlib.contextmanager
def my_context():
    """
    docstring
    """
    # Add any setup code you need.
    yield
    # Add any teardown code you need.
```

1. Define a function.
2. (optional) Add any setup code your context needs.
3. Use the `yield` keyword.
4. (optional) Add any teardown code your context needs.
5. Add the `@contextlib.contextmanager` decorator.

#### The `yield` Keyword (Demonstration)

A context manager function is technically a generator that returns a single value.

The ability for a function to yield control and know that it will get to finish running later is what makes context managers so useful.

This demonstrates a simple context manager that returns 42.

In [ ]:
@contextlib.contextmanager
def my_context():
    print("hello")
    yield 42
    print("goodbye")

with my_context() as foo:
    print("foo is {}".format(foo))

#### Setup and Teardown (Demonstration)

This context manager provides a connection to a database. (This is not a working example. I have changed variable names.)

```python
@contextlib.contextmanager
def get_conn(url):
    # Set up the database connection.
    conn = postgres.connect(url)
    yield conn
    conn.disconnect()

url = "http://datacamp.com/data"
with get_conn(url) as conn:
    course_list = conn.execute(
        "SELECT * FROM courses"
    )
```

See [Introduction to Databases in Python](../Introduction%20to%20Databases%20in%20Python/Introduction%20to%20Databases%20in%20Python.ipynb) for examples of using a context manager to connect to a SQLite database.

#### Yielding None (Demonstration)

`in_dir()` is a context manager that changes the current working directory to a specific path and then changes it back after the context block is done. It does not need to return anything with its "yield" statement.

```python
@contextlib.contextmanager
def in_dir(path):
    """
    During setup, save the old working directory and change the working
    directory to the new path.
    
    Yields:
        None
    
    During teardown, change the working directory back to the old path.
    """
    old_dir = os.getcwd()
    os.chdir(path)
    yield
    os.chdir(old_dir)

with in_dir("/data/project_1/"):
    project_files = os.listdir()
```

#### Create a `timer()` Context Manager (Exercise)

Write a context manager that can be used to time how long a function takes to run.

In [ ]:
@contextlib.contextmanager
def timer():
    """
    Time the execution of a context block.
    
    Yields:
        None
    """
    start = time.time()
    yield
    end = time.time()
    print("Elapsed: {:.2f} s".format(end - start))

with timer():
    print("This should take approximately 0.25 seconds.")
    time.sleep(0.25)

#### A Read-Only `open()` Context Manager (Exercise)

Create a context manager that will only open a file for reading.

In [ ]:
@contextlib.contextmanager
def open_read_only(filepath):
    """
    Open a file in read-only mode.
    
    Args:
        filepath (str): The path of the file to read
    
    Yields:
        a file object
    """
    read_only_file = open(filepath, mode="r")
    yield read_only_file
    read_only_file.close()

with open_read_only("my_file.txt") as my_file:
    print(my_file.read())

### Advanced Topics

#### Nested Contexts (Demonstration)

Write a function that copies a file line by line to a new file. This enables copying files that are too large to hold in memory.

In [ ]:
def copyfile(src, dst):
    """
    Copy the contents of one file to another.
    
    Args:
        src (str): Path of the source file to be copied
        dst (str): Path of the destination file to be written
    """
    with open(src, "r") as f_src:
        with open(dst, "w") as f_dst:
            # Copy line by line to accommodate very large files.
            for line in f_src:
                f_dst.write(line)

copyfile("my_file.txt", "my_copied_file.txt")

#### Handling Errors (Demonstration)

This is an example of how to set up your code to handle any errors and still tear down the context.

```python
@contextlib.contextmanager
def get_printer(ip):
    p = connect_to_printer(ip):
        p = connect_to_printer(ip)
        try:
            yield
        finally:
            p.disconnect()
            print("disconnected from printer")

doc = {"text": "This is my text"}
with get_printer("10.0.34.111") as printer:
    # Note the use of "txt" instead of "text" here, which raises
    # a KeyError.
    # The output looks like:
    # disconnected from printer
    # Traceback (most recent call last):
    #   File "<stdin>", line 1, in <module>
    #     printer.print_page(doc["txt"])
    # KeyError: "txt"
    printer.print_page(doc["txt"])
```

#### Context Manager Patterns

From Dave Brandsema's talk at PyCon 2012: https://youtu.be/cSbD5SKwak0?t=795.

|      |       |
| :--- | :---- |
| Open | Close |
| Lock | Release |
| Change | Reset |
| Enter | Exit |
| Start | Stop |
| Setvup | Tear down |
| Connect | Disconnect |

#### Context Manager Use Cases (Exercise)

Which of the following would or would not be a good opportunity to use a context manager?

1. A function that starts a timer that keeps track of how long some block of code takes to run. (Yes)
2. A function that prints all of the prime numbers between 2 and some value of `n`. (No)
3. A function that connects to a smart thermostat so that it can be programmed remotely. (Yes)
4. A function that prevents multiple users from updating an online spreadsheet at the same time by locking access to the spreadsheet before every operation. (Yes)

Example (2) above is a generator, not a context manager. "While you might be able to do this with a context manager, it would make much more sense just to do it with a normal function."

#### Using a Stock Price Simulator

This uses the CONNECT/DISCONNECT and OPEN/CLOSE context manager patterns.

I attempted to use `inspect.getsource()` to get the source code for the `stock` function and the `MockStock` function, but `MockStock` was a builtin. This prevented me from building a working version of this exercise. There was not a MockStock package at https://pypi.org.

```python
# Use the stock("NVDA") context manager to obtain ten stock prices
# and write them to a file.
with stock("NVDA") as nvda:
    with open("NVDA.txt", "w") as f_out:
        for _ in range(10):
            value = nvda.prince()
            print("Logging ${:.2f} for NVDA".format(value))
            f_out.write(":.2f}\n".format(value))

# import inspect
# inspect.getsource(stock)
@contextlib.contextmanager
def stock(symbol):
    base = 140.00
    scale = 1.0
    mock = MockStock(base, scale)
    print('Opening stock ticker for {}'.format(symbol))
    yield mock
    print('Closing stock ticker')

# inspect.getsource(MockStock)
# TypeError: <class '__main__.MockStock'> is a built-in class
```

This was the output in the console:

```
Opening stock ticker for NVDA
Logging $139.50 for NVDA
Logging $139.54 for NVDA
Logging $139.61 for NVDA
Logging $139.65 for NVDA
Logging $139.72 for NVDA
Logging $139.73 for NVDA
Logging $139.80 for NVDA
Logging $139.78 for NVDA
Logging $139.73 for NVDA
Logging $139.64 for NVDA
Closing stock ticker
```

#### Changing the Working Directory (Exercise)

This is an interesting exercise because I have encountered this problem before.

>You are using an open-source library that lets you train deep neural networks on your data. Unfortunately, during training, this library writes out checkpoint models (i.e., models that have been trained on a portion of the data) to the current working directory. You find that behavior frustrating because you don't want to have to launch the script from the directory where the models will be saved.

>You decide that one way to fix this is to write a context manager that changes the current working directory, lets you build your models, and then resets the working directory to its original location. You'll want to be sure that any errors that occur during model training don't prevent you from resetting the working directory to its original location.

This is an example of the CHANGE/RESET pattern.

```python
def in_dir(directory):
    """
    Change current working directory to `directory`,
    allow the user to run some code, and change back.

    Args:
        directory (str): The path to a directory to work in.
    """
    current_dir = os.getcwd()
    os.chdir(directory)

    # Add code that lets you handle errors
    try:
        yield
    # Ensure the directory is reset,
    # whether there was an error or not
    finally:
        os.chdir(current_dir)
```


## Decorators

This section looks at functions as objects, scope, closures, and decorators.

### Functions as Objects

#### Functions as Variables (Demonstration)

Functions are objects that can be stored as variables.

In [ ]:
# Assign functions to variables and use the variables.
def my_function():
    print("Hello")

x = my_function
x()

PrintyMcPrintFace = print
PrintyMcPrintFace("Python is awesome!")

list_of_functions = [my_function, open, print]
list_of_functions[2]("I am printing with an element of a list!")

dict_of_functions = {
    "func1": my_function,
    "func2": open,
    "func3": print,
}
dict_of_functions["func3"]("I am printing with an element of a dict!")

#### Functions as Arguments (Demonstration)

In [ ]:
def has_docstring(func):
    """
    Check to see if func has a docstring.
    
    Args:
        func (callable): A function.
    
    Returns:
        bool
    """
    return func.__doc__ is not None

print(has_docstring(print))
print(has_docstring(x))

def no():
    return 42

def yes():
    """
    Return the value 42.
    """
    return 42

print(has_docstring(no))
print(has_docstring(yes))

#### Defining a Function Inside Another Function (Demonstration)

In [ ]:
def foo():
    x = [3, 6, 9]
    
    # Inner, nested, helper, or child function.
    def bar(y):
        print(y)
    
    for value in x:
        bar(value)

foo()

# Start with this function.
def bar(x, y):
    if x > 4 and x < 10 and y > 4 and y < 10:
        print(x * y)

bar(1, 4)
bar(5, 7)

# Use a nested function to make this clearer.
def bar2(x, y):
    def in_range(v):
        return v > 4 and v < 10
    if in_range(x) and in_range(y):
        print(x * y)

bar2(1, 4)
bar2(5, 7)

#### Functions as Return Values (Demonstration)

In [ ]:
def get_function():
    def print_me(s):
        print(s)
    return print_me

new_func = get_function()
new_func("This is a sentence.")

#### Building a Command Line Data App (Exercise)

In [ ]:
"""
The code for load_data and get_user_input was obtained with the following
commands:

    import inspect
    inspect.getsource(load_data)
    inspect.getsource(get_user_input)
    inspect.getsource(mean)
    inspect.getsource(std)
    inspect.getsource(maximum)
    inspect.getsource(minimum)
"""

def load_data():
    df = pd.DataFrame()
    df['height'] = [72.1, 69.8, 63.2, 64.7]
    df['weight'] = [198, 204, 164, 238]
    return df

def get_user_input(prompt='Type a command: '):
    command = random.choice(['mean', 'std', 'minimum', 'maximum'])
    print(prompt)
    print('> {}'.format(command))
    return command

def mean(data):
    print(data.mean())

def std(data):
    print(data.std())

def minimum(data):
    print(data.min())

def maximum(data):
    print(data.max())

# Create a dict that maps a menu choice to a function.
function_map = {
    'mean': mean,
    'std': std,
    'minimum': minimum,
    'maximum': maximum
}
# Load the data into a DataFrame.
data = load_data()
print(data)
print()

# Use the user's choice of function to call the desired function.
func_name = get_user_input()
function_map[func_name](data)

#### Check for Function Docstrings (Exercise)

Determine whether a function has a docstring.

In [ ]:
# I used inspect.getsource to obtain the code for the
# has_docstring and load_and_plot_data functions.
def has_dostring(func):
    """
    Check to see if func has a docstring.
    
    Args:
        func (callable): A function.
        
    Returns:
        bool
    """
    return func.__doc__ is not None

def load_and_plot_data(filename):
    """
    Load a data frame and plot each column.
    
    Args:
        filename (str): Path to a CSV file of data.
    
    Returns:
        pandas.DataFrame
    """
    df = pd.load_csv(filename, index_col=0)
    df.hist()
    return df

def as_2D(arr):
    """
    Reshape an array to 2 dimensions
    """
    return np.array(arr).reshape(1, -1)

def log_product(arr):
    return np.exp(np.sum(np.log(arr)))

# Exercise code.
# I revised this code to get rid of repeated code.
def report_has_docstring(func):
    ok = has_docstring(func)
    if not ok:
        print("{} doesn't have a docstring!".format(func.__name__))
    else:
        print("{} looks OK.".format(func.__name__))

report_has_docstring(load_and_plot_data)
report_has_docstring(as_2D)
report_has_docstring(log_product)

#### Returning Functions for a Math Game (Exercise)

Use nested functions.

In [ ]:
# Supply functions for a math game.
def create_math_function(func_name):
    if func_name == 'add':
        def add(a, b):
            return a + b
        return add
    elif func_name == 'subtract':
        # Define the subtract() function
        def subtract(a, b):
            return a - b
        return subtract
    else:
        print("I don't know that one")
    
add = create_math_function('add')
print('5 + 2 = {}'.format(add(5, 2)))

subtract = create_math_function('subtract')
print('5 - 2 = {}'.format(subtract(5, 2)))

### Scope

The Python interpreter applies the following rules in order when looking for a name in a function:

- Search local scope, which includes arguments and any variables defined inside the function
- If there is a parent scope, search the nonlocal scope, the scope of the parent function
- If the name isn't found, search the global scope
- If the name isn't found, search the builtins

#### The `global` Keyword (Example)

In the Python documentation, see [the global statement](https://docs.python.org/3/reference/simple_stmts.html#the-global-statement).

How to modify a global variable from within a function:

In [ ]:
print("Local scope:")
x = 7
print(x)

def foo():
    # Create x in local scope.
    x = 42
    print(x)

foo()
# The x in the global scope was unaffected.
print(x)

print()
print("Global scope:")
x = 7
print(x)

def foo():
    # Modify x in global scope.
    global x
    x = 42
    print(x)

foo()
# The x in the global scope was modified.
print(x)

#### The `nonlocal` Keyword (Example)

In the Python documentation, see [the nonlocal statement](https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement).

How to modify a nonlocal variable within a function:

In [ ]:
print("Global scope:")
x = 7
print(x)

def foo():
    # Create x in local scope.
    print("Nonlocal scope")
    x = 10
    print(x)

    def bar():
        print("Local scope")
        x = 200
        print(x)
    
    bar()
    print("Nonlocal scope")
    print(x)
    
foo()
print("Global scope")
print(x)

print()
print("Global scope:")
x = 7
print(x)

def foo():
    # Create x in local scope.
    print("Nonlocal scope")
    x = 10
    print(x)

    def bar():
        nonlocal x
        print("Modify x in Nonlocal scope")
        x = 200
        print(x)
    
    bar()
    print("Nonlocal scope")
    print(x)
    
foo()
print("Global scope")
print(x)


#### Understanding Scope (Exercise)

What four values does this script print?

In [ ]:
# This should print 50, 30, 100, 30.
x = 50

def one():
    x = 10

def two():
    global x
    x = 30
    
def three():
    x = 100
    print(x)

for func in (one, two, three):
    func()
    # This prints the global x without modifying it.
    print(x)

#### Modifying Variables Outside Local Scope (Exercise)

Update `call_count` from within the function.

In [ ]:
call_count = 0

def my_function():
    # Update call_count in global scope.
    global call_count
    call_count += 1
    
    print("You've called my_function() {} time{}.".format(
        call_count,
        "" if call_count == 1 else "s"))
    
for _ in range(20):
    my_function()

Update `file_contents` from within `save_contents`.

In [ ]:
def read_files():
    file_contents = None
  
    def save_contents(filename):
        # Modify nonlocal file_contents.
        nonlocal file_contents
        if file_contents is None:
            file_contents = []
        with open(filename) as fin:
            file_contents.append(fin.read())
      
    for filename in ['1984.txt', 'MobyDick.txt', 'CatsEye.txt']:
        save_contents(filename)
    
    return file_contents

print('\n'.join(read_files()))

Update `done` from within `check_is_done`.

In [ ]:
def wait_until_done():
    def check_is_done():
        # Modify global done.
        global done
        if random.random() < 0.1:
            done = True
      
    while not done:
        check_is_done()

done = False
wait_until_done()

print('Work done? {}'.format(done))

### Closures

>A closue in Python is a tuple of variables that are no longer in scope but that a function needs in order to run.

>When `foo()` returned the new `bar` function, Python helpfully attached any nonlocal variable that `bar` was going to need to the function object. Those variables get stored in a tuple in the `__closure__` attribute of the function. The closure for `func` has one variable, and you can view the value of that variable by accessing the `cell_contents` of the item.

See also the documentation for the data model at https://docs.python.org/3/reference/datamodel.html.

#### Attaching Nonlocal Variables to Nest Functions (Demonstration)

In [ ]:
def foo():
    a = 5
    def bar():
        print(a)
    return bar

# func is bar, which needs access to a in foo.
func = foo()
func()

# View the closure attributes of func.
print()
print(type(func.__closure__))
print(len(func.__closure__))
print(func.__closure__)
print(type(func.__closure__[0]))
print(func.__closure__[0])
print(func.__closure__[0].cell_contents)
print(inspect.getclosurevars(func))

#### Closures and Deletion (Demonstration)

Even though we can delete `x`, the value persists in the `__closure__` attribute associated with function `my_func`. (That's because argument `value` in function `foo` is a nonlocal variable for function `bar`.)

In [ ]:
x = 25

def foo(value):
    def bar():
        print(value)
    return bar

my_func = foo(x)
my_func()
del(x)
my_func()

# View the closure attributes of my_func.
print()
print(type(my_func.__closure__))
print(len(my_func.__closure__))
print(my_func.__closure__)
print(type(my_func.__closure__[0]))
print(my_func.__closure__[0])
print(my_func.__closure__[0].cell_contents)
print(inspect.getclosurevars(my_func))

#### Closures and Overwriting (Demonstration)

We can overwrite `x`, yet the value persists in the closure.

In [ ]:
x = 25

def foo(value):
    def bar():
        print(value)
    return bar

# Overwrite x. Here, x becomes bar, and bar() prints the
# value associated with the closure.
x = foo(x)
x()

print(len(x.__closure__))
print(x.__closure__[0].cell_contents)

#### Key Concepts for Decorators

- A _nested function_ is a function defined inside another function.
- A _nonlocal variable_ is a variable defined in the parent function that is used by the nested function.
- A closure is nonlocal variables attached to a returned function.

In [ ]:
def parent(arg_1, arg_2):
    # From child's point of fiel, `arg_1`, `arg_2`, `value`, and `my_dict`
    # are nonlocal variables.
    value = 22
    my_dict = {"chocolate": "yummy"}
    
    def child():
        print(2 * value)
        print(my_dict["chocolate"])
        print(arg_1 + arg_2)
        
    return child

new_function = parent(3, 4)
new_function()

print()
print([cell.cell_contents for cell in new_function.__closure__])
print(inspect.getclosurevars(new_function))

#### Decorators

Decorators use:

- functions as objects
- nested functions
- nonlocal scope
- closures

#### Checking for Closure (Exercise)

In [ ]:
# Create a closure.
def return_a_func(arg1, arg2):
    def new_func():
        print("arg1 was {}".format(arg1))
        print("arg2 was {}".format(arg2))
    return new_func

# Use the __closure__ attribute to show that the nonlocal variables are in the closure.
my_func = return_a_func(2, 17)
print(my_func.__closure__ is not None)

# Show that there are two variables in the closure.
print(len(my_func.__closure__) == 2)

# Get the values of the variables in the closure.
closure_values = [
    my_func.__closure__[i].cell_contents for i in range(2)
]
print(closure_values == [2, 17])
# I would use the following list comprehension without needing an index:
closure_values = [cell.cell_contents for cell in my_func.__closure__]
print(closure_values)

#### Closures Keep Your Values Safe (Exercise)

Overwrite or delete a variable (here, a function). The nested function retains its values.

In [ ]:
def my_special_function():
    print('You are running my_special_function()')

def get_new_func(func):
    def call_func():
        func()
    return call_func

new_func = get_new_func(my_special_function)
new_func()

# The order of the next steps was important.
# Overwite my_special_function.
my_special_function = get_new_func(my_special_function)
my_special_function()

# Delete my_special_function.
del my_special_function
new_func()

# Redefine my_special_function() to just print "hello"
def my_special_function():
    print('hello')
new_func()

You could run into memory issues if you wound up adding a very large array or object to the closure.

### Decorators

#### Building the `double_args` Decorator (Demonstration)

The `double_args` decorator doubles the value of each argument passed to the function. When we're done, `multiply` equals the value returned by calling `double_args` with `multiply` as the only argument.

This is where we're headed:

```python
@double_args
def multiply(a, b):
    return a * b
multiply(1, 5) # Returns (2 * 1) * (2 * 5) = 20
```

In [ ]:
# Step 1. Create the decorator function with no modification of the original
# function.
def multiply(a, b):
    return a * b
def double_args(func):
    return func
new_multiply = double_args(multiply)
print(new_multiply(1, 5))
print(multiply(1, 5))

In [ ]:
# Step 2. Create and return a wrapper function in double_args.
def double_args(func):
    # Define a new function that we can modify.
    def wrapper(a, b):
        return func(a, b)
    
    return wrapper
new_multiply = double_args(multiply)
print(new_multiply(1, 5))

In [ ]:
# Step 3. Modify the wrapper function.
def double_args(func):
    def wrapper(a, b):
        # Double the value of each argument when calling func.
        return func(2 * a, 2 * b)
    
    return wrapper
new_multiply = double_args(multiply)
print(new_multiply(1, 5))

In [ ]:
# Step 4. Overwrite multiply with the new function.
# Python stores func in double_args.__closure__.
def multiply(a, b):
    return a * b
def double_args(func):
    def wrapper(a, b):
        return func(a * 2, b * 2)
    return wrapper
multiply = double_args(multiply) # multiply is now wrapper.
print(multiply(1, 5))
# Examine the closure.
print(multiply.__closure__[0].cell_contents)

In [ ]:
# Step 5. Use the decorator syntax.
def double_args(func):
    def wrapper(a, b):
        return func(a * 2, b * 2)
    return wrapper

@double_args
def multiply(a, b):
    return a * b

print(multiply(1, 5))

#### Use Decorator Syntax (Exercise)

Decorate a function using code. Decorate the function using decorator syntax.

In [ ]:
# This is code we weren't given for the decorator function.
def print_args(func):
    sig = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = sig.bind(*args, **kwargs).arguments
        str_args = ', '.join(['{}={}'.format(k, v) for k, v in bound.items()])
        print('{} was called with {}'.format(func.__name__, str_args))
        return func(*args, **kwargs)
    return wrapper

# Step 1: This is the exercise code.
def my_function(a, b, c):
    print(a + b + c)

# Decorate my_function() with the print_args decorator.
my_function = print_args(my_function)

my_function(1, 2, 3)

# Step 2:
# Decorate my_function with the print_args decorator
# using decorator syntax.
@print_args
def my_function(a, b, c):
    print(a + b + c)

my_function(1, 2, 3)

#### Defining a Decorator (Exercise)

In [ ]:
# Create a decorator function and use it.
def print_before_and_after(func):
    def wrapper(*args):
        print('Before {}'.format(func.__name__))
        func(*args)
        print('After {}'.format(func.__name__))
    return wrapper

@print_before_and_after
def multiply(a, b):
    print(a * b)

multiply(5, 10)

## More on Decorators

### Real-World Examples

#### Timer Decorator (Demonstration)

In [ ]:
def timer(func):
    """
    A decorator that prints how long a function took to run.
    
    Args:
        func (callable): The function being decorated.
    
    Returns:
        callable: The decorated function (the wrapper function).
    """
    def wrapper(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        t_total = time.time() - t_start
        print("{} took {}s.".format(func.__name__, t_total))
        return result
    return wrapper

@timer
def sleep_n_seconds(n):
    time.sleep(n)

# Be patient!
sleep_n_seconds(5)
sleep_n_seconds(10)

#### Memoizing (Demonstration)

In [ ]:
# The code presented in the demonstration didn't work.
# Instead of using (args, kwargs) as the key,
# I created a string key, following the recommendation in
# https://stackoverflow.com/questions/28145601/how-can-i-write-python-decorator-for-caching.
# An alternative is to use the functools.cache decorator (see below).
def memoize(func):
    """
    Store the results of the decorated function for fast lookup.
    """
    # Store results in a dict that maps arguments to results.
    cache = {}
    def wrapper(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return wrapper

@memoize
def slow_function(a, b):
    print("Sleeping...")
    time.sleep(5)
    return a + b

# Performs the calculation and returns the result after cacheing it.
print(slow_function(3, 4))
# Uses cached value, so returns faster the second time.
print(slow_function(3, 4))
# Note that the name of the function gets changed.
print(slow_function.__name__)

#### `functools.wrap` Decorator (Extra)

Use this decorator to preserve the name and docstring of the wrapped function. See https://docs.python.org/3/library/functools.html#functools.wraps.

In [ ]:
# Demonstrate use of functools.wraps.
def memoize(func):
    """
    Store the results of the decorated function for fast lookup.
    """
    # Store results in a dict that maps arguments to results.
    cache = {}
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return wrapper

@memoize
def slow_function(a, b):
    print("Sleeping...")
    time.sleep(5)
    return a + b

# Performs the calculation and returns the result after cacheing it.
print(slow_function(3, 4))
# Uses cached value, so returns faster the second time.
print(slow_function(3, 4))
# Note that the name of the function gets changed.
print(slow_function.__name__)

#### `functools.cache` Decorator (Extra)

Python has a `functools.cache` decorator. See https://docs.python.org/3/library/functools.html#functools.cache.

In [ ]:
@functools.cache
def slow_function(a, b):
    print("Sleeping...")
    time.sleep(5)
    return a + b

# Performs the calculation and returns the result after cacheing it.
print(slow_function(3, 4))
# Uses cached value, so returns faster the second time.
print(slow_function(3, 4))

#### When to Use Decorators

Use a decorator to add common behavior to multiple functions (don't repeat yourself).

#### Print the Return Type Decorator (Exercise)

Write a decorator, `print_return_type`, that prints the type of the variable that gets returned from every call of any function it is decorating.

In [ ]:
def print_return_type(func):
    # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
            func.__name__, type(result)
        ))
        return result
    # Return the decorated function
    return wrapper
  
@print_return_type
def foo(value):
    return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

#### Counter Decorator (Exercise)

Write a decorator that adds a counter to each function that you decorate. You could use this information in the future to determine whether there are sections of code that you could remove because they are no longer being used by the app.

In [ ]:
def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    wrapper.count = 0
    # Return the new decorated function
    return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
    print('calling foo()')

foo()
foo()

print('foo() was called {} times.'.format(foo.count))

### Decorators and Metadata

#### Obscuring of Metadata (Demonstration)

A problem with decorators is that they obscure metadata, such as the name and docstring of the wrapped function.

In [ ]:
# Define a function and print its docstring, name, and default arguments.
def sleep_n_seconds(n=10):
    """
    Pause processing for n seconds.
    
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

print("Before decoration:")
print(sleep_n_seconds.__doc__)
print(sleep_n_seconds.__name__)
print(sleep_n_seconds.__defaults__)
print()

# Decorate sleep_n_seconds with the timer decorator and print the metadata.
@timer
def sleep_n_seconds(n=10):
    """
    Pause processing for n seconds.
    
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

print("After decoration with timer:")
print(sleep_n_seconds.__doc__)
print(sleep_n_seconds.__name__)
print(sleep_n_seconds.__defaults__)
print()

del timer
del sleep_n_seconds

# Use the functools.wraps decorator to avoid this problem.
def timer(func):
    """
    A decorator that prints how long a function took to run.
    
    Args:
        func (callable): The function being decorated.
    
    Returns:
        callable: The decorated function (the wrapper function).
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        t_total = time.time() - t_start
        print("{} took {}s.".format(func.__name__, t_total))
        return result
    
    return wrapper

@timer
def sleep_n_seconds(n=10):
    """
    Pause processing for n seconds.
    
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

print("After decoration with timer decorated with functools.wraps:")
print(sleep_n_seconds.__doc__)
print(sleep_n_seconds.__name__)
# For me, withh Python 3.11.2, this doesn't print the default arguments
# for sleep_n_seconds. This is a feature, not a bug.
# See https://bugs.python.org/issue41232.
# See https://docs.python.org/3/library/functools.html#functools.update_wrapper
print(sleep_n_seconds.__defaults__)
print()

# Show access to the original function.
print("Get attributes from __wrapped__ attribute:")
print(sleep_n_seconds.__wrapped__)
print(sleep_n_seconds.__wrapped__.__doc__)
print(sleep_n_seconds.__wrapped__.__name__)
print(sleep_n_seconds.__wrapped__.__defaults__)

#### Preserving Docstrings When Decorating Functions (Exercise)

In [ ]:
# Create the decorator function.
def add_hello(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """
        Print 'hello' and then call the decorated function.
        """
        print('Hello')
        return func(*args, **kwargs)
    return wrapper

# Decorate print_sum() with the add_hello() decorator
@add_hello
def print_sum(a, b):
    """
    Adds two numbers and prints the sum.
    """
    print(a + b)

print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

#### Measuring Decorator Overhead (Exercise)

Compare performance of the wrapped function with the performance of the unwrapped function using the `__wrapped__` attribute of the decorated function.

In [ ]:
# Add code that supports this exercise, obtained using inspect.getsource().
def check_inputs(a, *args, **kwargs):
    for value in a:
        time.sleep(0.01)
    print('Finished checking inputs')

def check_outputs(a, *args, **kwargs):
    for value in a:
        time.sleep(0.01)
    print('Finished checking outputs')

def check_everything(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        check_inputs(*args, **kwargs)
        result = func(*args, **kwargs)
        check_outputs(result)
        return result
    return wrapper

# Exercise code.
@check_everything
def duplicate(my_list):
    """
    Return a new list that repeats the input twice.
    """
    return my_list + my_list

t_start = time.time()
duplicated_list = duplicate(list(range(50)))
t_end = time.time()
decorated_time = t_end - t_start

t_start = time.time()
# Call the original function instead of the decorated one
duplicated_list = duplicate.__wrapped__(list(range(50)))
t_end = time.time()
undecorated_time = t_end - t_start

print('Decorated time: {:.5f}s'.format(decorated_time))
print('Undecorated time: {:.5f}s'.format(undecorated_time))

### Decorators that Take Arguments

### Timeout(): A Real World Example

## Equations in Jupyter Lab

Jupyter Lab uses MathJax to render math created using LaTeX symbols. See https://towardsdatascience.com/how-to-convert-jupyter-notebooks-into-pdf-5accaef3758.

When I tested printing the notebook to PDF from within Jupyter Lab, the LaTeX code, not the equation, appeared.

The Jupyter Lab menu includes File > Save and Export Notebook As... > PDF. Once I had installed MacTeX and pandoc using Homebrew, this worked nicely to create a PDF file in the working directory. The file contained the math equation in the cell below.

The `jupyter nbconvert` command can create a PDF file from a Jupyter notebook.

```
$ cd ~/src/conradhalling/datacamp
$ jupyter nbconvert --to pdf Writing\ Functions\ in\ Python/Writing\ Functions\ in\ Python.ipynb
	[NbConvertApp] Converting notebook Writing Functions in Python/Writing Functions in Python.ipynb to pdf
	/Users/halto/src/conradhalling/datacamp/venv/lib/python3.11/site-packages/nbconvert/utils/pandoc.py:51: RuntimeWarning: You are using an unsupported version of pandoc (3.1.1).
	Your version must be at least (1.12.1) but less than (3.0.0).
	Refer to https://pandoc.org/installing.html.
	Continuing with doubts...
	  check_pandoc_version()
	[NbConvertApp] Writing 115530 bytes to notebook.tex
	[NbConvertApp] Building PDF
	[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
	[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
	[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
	[NbConvertApp] PDF successfully created
	[NbConvertApp] Writing 134473 bytes to Writing Functions in Python/Writing Functions in Python.pdf
```

\begin{equation}
\hat{Y} = \hat{\beta}\_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j}
\end{equation}